In [ ]:
%pip install ipympl -q
%pip install numpy -q
%pip install wheel

%pip install keras -q
%pip install keras-cv -q

%pip install matplotlib -q

In [1]:
location_to_saved_model: str = '/home/spectre/environment_model.keras'

In [2]:
import tensorflow

gpus = tensorflow.config.list_physical_devices(
    'GPU'
)

2024-01-23 19:56:48.820115: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-23 19:56:48.868285: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-23 19:56:48.868312: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-23 19:56:48.869927: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-23 19:56:48.879624: I tensorflow/core/platform/cpu_feature_guar

In [3]:
nvidia_maximum_limit_for_gpu_memory: int = 5925

if gpus:
    try:
        for gpu in gpus:
            tensorflow.config.set_logical_device_configuration(
                gpu, 
                [
                    tensorflow.config.LogicalDeviceConfiguration(
                        memory_limit=nvidia_maximum_limit_for_gpu_memory
                    )
                ]
            )
    except RuntimeError as e:
        print(e)

location_to_dataset: str = '/tmp/dataset'
name_of_location: str = 'LocationDataset'

In [4]:
from os.path import isdir

if not isdir(
    location_to_dataset
):
    !git clone https://github.com/TargetSpectre/Location-Identification-DataSet.git {name_of_location} 
    !mv ./{name_of_location}/DataSet {location_to_dataset}

!ls {location_to_dataset}

Anubis	Default  Dust-II  Mirage  Vertigo


# 

In [4]:
from os import listdir
from os.path import join

labels: list = list()
found_dirs = listdir(
    location_to_dataset
)

for idx in range(
    len(
        found_dirs
    )
):
    selected = found_dirs[idx]
    labels.append(
        selected
    )

    found_dirs[idx] = join(
        location_to_dataset, 
        selected
    )

labels.sort()

print(
    labels
)

['Anubis', 'Default', 'Dust-II', 'Mirage', 'Vertigo']


In [6]:
from random import SystemRandom
from keras.utils import image_dataset_from_directory

In [42]:
batch_size: int = 2
buffer_size: int = 10

number_of_training_iterations: int = 5  

vision_width: int = 960
vision_height: int = 540

maximum_value: int = 65535

In [43]:
from keras import layers
from keras.models import Model, load_model
from keras.optimizers import Adam

In [45]:
model = load_model(
    location_to_saved_model
)

model.summary()

model.compile(
    optimizer   = Adam(
        learning_rate   = 0.001,
        amsgrad         = True,
        global_clipnorm = 10.0,
        use_ema         = True
    ),
    loss        = 'categorical_crossentropy',
    metrics     = ['accuracy']
)

Model: "location_identifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 540, 960, 3)]     0         
                                                                 
 conv2d_44 (Conv2D)          (None, 540, 960, 4)       52        
                                                                 
 conv2d_45 (Conv2D)          (None, 540, 960, 4)       68        
                                                                 
 conv2d_46 (Conv2D)          (None, 540, 960, 4)       68        
                                                                 
 conv2d_47 (Conv2D)          (None, 540, 960, 4)       68        
                                                                 
 batch_normalization_12 (Ba  (None, 540, 960, 4)       16        
 tchNormalization)                                               
                                               

In [10]:
from keras.callbacks \
    import EarlyStopping

callbacks: list = [
    EarlyStopping(
        monitor                 = 'accuracy',
        min_delta               = 0,
        patience                = 0,
        mode                    = 'max',
        restore_best_weights    = True,
        start_from_epoch        = 2
    )
]


In [29]:
def generate_random_int16_number() -> int:
    global maximum_value
    return SystemRandom().randint(
            1, 
            maximum_value
    )

In [31]:
def generate_training_dataset():
    global batch_size, vision_height, vision_width
    
    return image_dataset_from_directory(
        location_to_dataset,
        labels                  = 'inferred',
        label_mode              = 'categorical',
        color_mode              = 'rgb',
        batch_size              = batch_size,
        image_size              = (
            vision_height, 
            vision_width
        ),
        shuffle                 = True,
        crop_to_aspect_ratio    = True,
        subset                  = 'training',
        validation_split        = 0.25,
        seed                    = generate_random_int16_number()
    )

In [32]:
def generate_validation_dataset():
    global batch_size, vision_height, vision_width
    return image_dataset_from_directory(
        location_to_dataset,
        labels                  = 'inferred',
        label_mode              = 'categorical',
        color_mode              = 'rgb',
        batch_size              = batch_size,
        image_size              = (
            vision_height, 
            vision_width
        ),
        shuffle                 = True,
        crop_to_aspect_ratio    = True,
        subset                  = 'validation',
        validation_split        = 0.25,
        seed                    = generate_random_int16_number()
    )

In [33]:
number_of_epochs: int = 10

In [49]:
def train():
    global callbacks, number_of_epochs
    
    training_dataset = generate_training_dataset()
    validation_dataset = generate_validation_dataset()

    history = model.fit(
        training_dataset.prefetch(
            buffer_size = buffer_size
        ), 
        epochs              = number_of_epochs, 
        validation_data     = validation_dataset.prefetch(
                                buffer_size = buffer_size
                              ),
        workers             = 12,
        callbacks           = callbacks,
        verbose             = 2
    )

In [50]:
for idx in range(
    number_of_training_iterations
):
    train()

model.save(
    location_to_saved_model,
    overwrite = True
)

Found 12300 files belonging to 5 classes.
Using 9225 files for training.
Found 12300 files belonging to 5 classes.
Using 3075 files for validation.
Epoch 1/10
4613/4613 - 222s - loss: 9.4888 - accuracy: 0.3456 - val_loss: 9.6813 - val_accuracy: 0.3580 - 222s/epoch - 48ms/step
Epoch 2/10
4613/4613 - 208s - loss: 9.4503 - accuracy: 0.3410 - val_loss: 9.6865 - val_accuracy: 0.3587 - 208s/epoch - 45ms/step
Epoch 3/10
4613/4613 - 210s - loss: 9.4608 - accuracy: 0.3457 - val_loss: 9.6760 - val_accuracy: 0.3584 - 210s/epoch - 45ms/step
Epoch 4/10
4613/4613 - 209s - loss: 9.4870 - accuracy: 0.3440 - val_loss: 9.6865 - val_accuracy: 0.3577 - 209s/epoch - 45ms/step
Found 12300 files belonging to 5 classes.
Using 9225 files for training.
Found 12300 files belonging to 5 classes.
Using 3075 files for validation.
Epoch 1/10
4613/4613 - 207s - loss: 9.4975 - accuracy: 0.3473 - val_loss: 9.7966 - val_accuracy: 0.3564 - 207s/epoch - 45ms/step
Epoch 2/10
4613/4613 - 207s - loss: 9.4975 - accuracy: 0.34